In [2]:
import psycopg2
import datetime

In [5]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [21]:
#geojson情報を作成する。
sql = "SELECT trakan2209.時刻,trakan2209.断面交通量,trakangeoload.multilinestring FROM trakangeoload "
sql += "INNER JOIN roadmap on roadmap.osm_id = trakangeoload.osm_id "
sql += "INNER JOIN trakanmap on trakanmap.gid = roadmap.gid "
sql += "INNER JOIN trakan2209 on trakan2209.計測地点番号 = trakanmap.地点番号 "
sql += "WHERE 時刻 < '2022-09-02 00:00:00' "
sql += "ORDER BY trakan2209.時刻 "

cur =connection.cursor()
cur.execute(sql)
keylist = []
for row in cur:
    keylist.append([str(row[0]), str(row[1]), str(row[2])])
cur.close()



In [9]:
#geojson情報を作成する。
sql = "SELECT times,a.osm_id,max(tbran),trakangeoload.multilinestring "
sql += " FROM kddi_1218_param_new_result_loadsummary  AS a "
sql += " INNER JOIN trakangeoload on a.osm_id = trakangeoload.osm_id  "
sql += " group BY times, a.osm_id,trakangeoload.multilinestring ORDER BY times, a.osm_id "
cur =connection.cursor()
cur.execute(sql)
keylist = []
for row in cur:
    keylist.append([str(row[0]), str(row[2]), str(row[3])])
cur.close()



In [ ]:
keylist

In [11]:
f = open('leafletTL.geojson', 'w')
find = 0
f.write('{\n')
f.write('    "type": "FeatureCollection",\n')
f.write('    "features": [\n')

wk = ""
for item in keylist:
        if find != 0:
            f.write('        ,\n')
        f.write('        {\n')
        f.write('           "type": "Feature",\n')
        f.write('           "properties": {\n')
        f.write('               "start": "')
        f.write(item[0])
        dte = datetime.datetime.strptime(item[0], '%Y-%m-%d %H:%M:%S')
        f.write('",\n')
        f.write('               "end": "')
        dte = dte + datetime.timedelta(minutes=15)
        f.write(str(dte))
        f.write('",\n')
        f.write('               "colordata":"')
        f.write(item[1])
        f.write('"\n')
        f.write('           },\n')
        f.write('           "geometry": {\n')
        f.write('               "type": "MultiLineString",\n')
        f.write('               "coordinates":\n')
        f.write('               ')
        f.write(item[2])
        f.write('           }\n')
        f.write('        }\n')
        find += 1
        wk = item[0]
f.write('    ]\n')
f.write('}\n')
f.close()


In [3]:
sttime = "2022-04-01 00:00:00"

f = open('leafletTL00-12.geojson', 'w')
find = 0
f.write('{\n')
f.write('    "type": "FeatureCollection",\n')
f.write('    "features": [\n')

wk = ""
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        break
        dte = dte + datetime.timedelta(days=i)
    for j in range(48):
        if find != 0:
            f.write('        ,\n')
        f.write('        {\n')
        f.write('           "type": "Feature",\n')
        f.write('           "properties": {\n')
        f.write('               "start": "')
        f.write(str(dte))
        f.write('",\n')
        f.write('               "end": "')
        dte = dte + datetime.timedelta(minutes=15)
        f.write(str(dte))
        f.write('"\n')
        f.write('           }\n')
        f.write('        }\n')
        find += 1
f.write('    ]\n')
f.write('}\n')
f.close()
